In [397]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [405]:
import lsst.afw.geom
from lsst.daf.persistence import Butler
from astropy import wcs
import astropy.units as u
from astropy.nddata import CCDData, VarianceUncertainty
import lsst.geom as geom
_scale = (1.0 * lsst.geom.arcseconds).asDegrees()

In [399]:
def computeSkySeperation(ra1, dec1, ra2, dec2):
    """Compute the local pixel scale conversion.

    Parameters
    ----------
    ra1 : `pandas.Series`
        Ra of the first coordinate in radians.
    dec1 : `pandas.Series`
        Dec of the first coordinate in radians.
    ra2 : `pandas.Series`
        Ra of the second coordinate in radians.
    dec2 : `pandas.Series`
        Dec of the second coordinate in radians.

    Returns
    -------
    dist : `pandas.Series`
        Distance on the sphere in radians.
    """
    deltaDec = dec2 - dec1
    deltaRa = ra2 - ra1
    return 2 * np.arcsin(
        np.sqrt(
            np.sin(deltaDec / 2) ** 2
            + np.cos(dec2) * np.cos(dec1) * np.sin(deltaRa / 2) ** 2))

Load HiTS2015 CI data from a ap_verify run.

In [400]:
b = Butler("/project/morriscb/src/ap_verify_ci_hits2015/testData/output")

In [401]:
diffIm = b.get("deepDiff_differenceExp", visit=419802, ccdnum=10)
diaSrc = b.get("deepDiff_diaSrc", visit=419802, ccdnum=10)

In [404]:
calibDiffIm = diffIm.getPhotoCalib().calibrateImage(diffIm.getMaskedImage())

In [403]:
np.all(np.isnan(ans.getArrays()[0]))

False

Get a cutout from the difference image.

In [28]:
cutout = diffIm.getCutout(diaSrc[0].getCoord(), geom.Extent2I(30, 30))

Create an empty astropy WCS.

In [741]:
cutoutWcs = wcs.WCS(naxis=2)

Using the local CDMatrix, create a astropy WCS centered at Ra=180 and Dec=0 on the sphere and at the centroid of the object on the CCD. The cutOutMinX, cutOutMinY are the so that the WCS displays properly within the cutout in DS9.

In [762]:
offset = 0
pixOffset = 1
cutOutMinX = (cutout.getBBox().minX - 1)
cutOutMinY = (cutout.getBBox().minY - 1)

In [763]:
cutoutWcs.array_shape = (cutout.getBBox().getWidth(), cutout.getBBox().getWidth())
cutoutWcs.wcs.crpix = [diaSrc[0].getCentroid()[0] - cutOutMinX,
                       diaSrc[0].getCentroid()[1] - cutOutMinY]
cutoutWcs.wcs.crval = [diaSrc[0].getRa().asDegrees(), diaSrc[0].getDec().asDegrees() + offset]
cutoutWcs.wcs.ctype = ["RA---TAN", "DEC--TAN"]
cdMatrix = np.degrees(np.array([[diaSrc[0]["base_LocalWcs_CDMatrix_1_1"], diaSrc[0]["base_LocalWcs_CDMatrix_1_2"]],
                                [diaSrc[0]["base_LocalWcs_CDMatrix_2_1"], diaSrc[0]["base_LocalWcs_CDMatrix_2_2"]]]))
cutoutWcs.wcs.cd = cdMatrix

In [764]:
cutoutWcs.array_shape

(30, 30)

Create a test point 1 pixel away in x and y and compare to.

In [765]:
pixCent = np.array([[diaSrc[0].getCentroid()[0] - cutOutMinX + pixOffset,
                     diaSrc[0].getCentroid()[1] - cutOutMinY + pixOffset]])
print(diaSrc[0].getCentroid()[0], diaSrc[0].getCentroid()[1])
deltaCent = (pixOffset, pixOffset)
print("stack LocalWcs:", np.dot(cdMatrix, deltaCent))
coordRad = np.radians(np.dot(cdMatrix, deltaCent))
print("dist:", 3600 * np.degrees(computeSkySeperation(coordRad[0], coordRad[1], 0, 0)))

979.0936279296875 71.04425811767578
stack LocalWcs: [ 7.30649567e-05 -7.30157686e-05]
dist: 0.3718608382769982


In [766]:
astropyCoord = cutoutWcs.wcs_pix2world(pixCent, 1)[0]
print("astropy Wcs:",
      cutoutWcs.wcs_pix2world(pixCent, 1)[0],
      cutoutWcs.wcs_pix2world([[diaSrc[0].getCentroid()[0] - cutOutMinX,
                                diaSrc[0].getCentroid()[1] - cutOutMinY]], 0)[0])
3600 * np.degrees(computeSkySeperation(np.radians(astropyCoord[0]), np.radians(astropyCoord[1]),
                                       diaSrc[0].getRa().asRadians(),
                                       diaSrc[0].getDec().asRadians() - np.radians(offset)))

astropy Wcs: [154.97480413  -5.93919559] [154.97480413  -5.93919559]


0.3718608383434003

In [767]:
print(diffIm.getWcs().pixelToSky(diaSrc[0].getCentroid()[0] + pixOffset,
                                 diaSrc[0].getCentroid()[1] + pixOffset).getRa().asDegrees(),
      diffIm.getWcs().pixelToSky(diaSrc[0].getCentroid()[0] + pixOffset,
                                 diaSrc[0].getCentroid()[1] + pixOffset).getDec().asDegrees())
diffIm.getWcs().pixelToSky(diaSrc[0].getCentroid()[0] + pixOffset,
                           diaSrc[0].getCentroid()[1] + pixOffset).separation(
    diffIm.getWcs().pixelToSky(diaSrc[0].getCentroid())).asArcseconds()

154.97480412582692 -5.939195586146784


0.3718609588644356

Looks good for a large number of digits. Units are in degrees.

Create a new CCDData object in astropy by stripping out the data from teh cutout.

In [768]:
154.97480413 - 154.97480412582692, -5.93919559 - -5.939195586146784

(4.173074330537929e-09, -3.853215524429743e-09)

In [769]:
calibDiffIm = diffIm.getPhotoCalib().calibrateImage(cutout.getMaskedImage())
allArrays = calibDiffIm.getArrays()
ccdData = CCDData(data=calibDiffIm.getImage().array,
                  uncertainty=VarianceUncertainty(calibDiffIm.getVariance().array),
                  flags=calibDiffIm.getMask().array,
                  wcs=cutoutWcs,
                  meta={"cutMinX": cutOutMinX,
                        "cutMinY": cutOutMinY,},
                  unit=u.nJy)

In [770]:
ccdData.write("/project/morriscb/testCutout.fits", overwrite=True)

Questions above: Is that all we need for the ccdData cutout? Do we want to calibrate the image before creating the stamp? Does it make sense that the way I've created the WCS (i.e. projects onto the the equator at RA=180 with the center of the ccd image being at the true xy position)? With this structure, points the chip will be absolute while on the sky they will only be relative. I can easily also make the WCS center be xy=(0, 0) and all positions relative to the detected object center. I'm not sure what would be best to define the CPix. The WCS is and object center is not the center of one of the pixels pixtured. The draw back is that the image as it were has no info about what what the absolute position of a pixel is. [answered]

In [771]:
ccdData.wcs.to_header()

WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =      15.093627929688 / Pixel coordinate of reference point            
CRPIX2  =      15.044258117676 / Pixel coordinate of reference point            
PC1_1   = -9.2055734733204E-08 / Coordinate transformation matrix element       
PC1_2   =  7.3157012447246E-05 / Coordinate transformation matrix element       
PC2_1   = -7.2992717769792E-05 / Coordinate transformation matrix element       
PC2_2   = -2.3050793857232E-08 / Coordinate transformation matrix element       
CDELT1  =                  1.0 / [deg] Coordinate increment at reference point  
CDELT2  =                  1.0 / [deg] Coordinate increment at reference point  
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CTYPE1  = 'RA---TAN'           / Right ascension, gnomonic projection           
CTYPE2  = 'DEC--TAN'        